### Загрузка данных для бухгалтеров и директоров

### Загрузка и подключение библиотек

In [13]:
%%capture
!pip install transformers
!pip install sentencepiece
!pip install umap-learn

In [14]:
%%capture
import gdown
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer
from itertools import groupby
import json

## Эмбеддинги и кластеризация

### Класс для получения эмбеддингов текста

In [15]:
class RuBertEmbedder:

    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
        self.model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

        if torch.cuda.is_available():
            self.model.cuda()

        self.device = self.model.device

    def embed_bert_cls(self, text):
        """Получение эмбеддингов из текста"""
        t = self.tokenizer(text.replace('\n', ''), padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()

    def encode_data(self, text_pool, data_column='content'):
        print("Encoding data...")
        """
            Вход:
                content - содержимое новости
                date - дата публикации
            Выход:
                embeddings_pool- list с эмбеддингами новостей
        """
        embeddings_pool = text_pool.apply(
            lambda x: self.embed_bert_cls(x[data_column]), axis=1
            )
        print("Encoding done!")
        return embeddings_pool

### Класс для кластеризации эмбеддингов (k-means)

In [16]:
class KMeansClustering:
    def __init__(self, text_pool, embeddings, clusters_num = None):
        """
            Класс для кластеризации эмбеддингов.
            На вход принмает информацию о новостях, эмбеддинги и количество кластеров.
            Если оно не задано, то задаётся эмпирически.
        """
        self.embeddings = embeddings
        self.clusters_num = clusters_num
        self.text_pool = text_pool

        if clusters_num is None:
            self.clusters_num = round((float(len(embeddings)))**(1/2.2)) # эвристика
        
    def clustering(self):
        """
            Функция возвращает выполняющая поиск центроид кластеров.
            Выход: 
                data - датафрейм с информацией о новостях, в который добавлены
            метки кластеров и эмбеддинги,
                centoids - центроиды кластеров
        """
        print("Clustering data...")
        kmeans = KMeans(n_clusters = self.clusters_num, random_state = 42).fit(self.embeddings.to_list())
        kmeans_labels = kmeans.labels_
    
        data = pd.DataFrame()
        data['title'] = self.text_pool['title']
        data['text'] = self.text_pool['content']
        data['label'] = kmeans_labels
        data['embedding'] = self.embeddings.to_list()
    
        centroids = kmeans.cluster_centers_

        print("Clustering done!")
        
        return data, centroids

## Тренды, инсайты, дайджест

### Тренды

In [17]:
def sort_and_remove_repeat(keys_weights):
        """
        Убираем повторения 
        Сортируем по второму аргументу массив вида [([x1], [y1]), ([x2], [y2]), ...], 
        где 
        * y2 - это частота
        * x1 - это ключевое слово
        """
        dict_keys = {}
        set_stop_words = set(['анализ и проектирование систем', \
                            'промышленное программирование', 'читальный зал', \
                            'разработка веб-сайтов', 'программирование микроконтроллеров'\
                            'системное программирование', 'ненормальное программирование',\
                            'мобильная разработка', 'разработка мобильных приложений',\
                            'будущее здесь', 'научно-популярное', 'платежи в интернет',\
                            'платежи с мобильного', 'разработка игр', 'монетизация игр', 'дизайн игр'])
        

        for i in range(len(keys_weights)):
            for j in range(len(keys_weights[i][0])):
                if keys_weights[i][0][j] in set_stop_words:
                    continue
                elif not (keys_weights[i][0][j] in dict_keys):
                    dict_keys[keys_weights[i][0][j]] = keys_weights[i][1][j]
                else:
                    dict_keys[keys_weights[i][0][j]] += keys_weights[i][1][j]

        dict_zip = list(zip(dict_keys.keys(), dict_keys.values()))

        return sorted(dict_zip, key=lambda tup: tup[1], reverse=True)


class KeyWordsExtractor:

    def __init__(self, model_name="0x7194633/keyt5-large"):
        self.model_name = model_name

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        if torch.cuda.is_available():
            self.model.cuda()

        self.device = self.model.device

    def generate(self, text,  **kwargs):
        """
        Производим генерацию keywords
        """

        inputs = self.tokenizer(text, return_tensors='pt').to(self.device)

        with torch.no_grad():
            hypotheses = self.model.generate(**inputs, num_beams=5, **kwargs)

        s = self.tokenizer.decode(hypotheses[0], skip_special_tokens=True)
        s = s.replace('; ', ';').replace(' ;', ';').lower().split(';')[:-1]
        gamma = 1
        s = [el for el, _ in groupby(s)]
        weights = [gamma**i for i in range(len(s))]

        return s, weights

    def get_keywords(self, set_of_articles, **kwargs):
        """
        Получаем отсортированные по частоте сгенерированные ключевые фразы из набора статей

        [(key_1, weight_1), (key_2, weight_2), ....]
        """

        keys_weights = []
        len_set = len(set_of_articles)

        for i in range(len_set):
            text = set_of_articles[i]
            keys_weights.append(self.generate(text, **kwargs))

        return sort_and_remove_repeat(keys_weights)
        

    def get_trends(self, set_of_articles, n=5, threshold=0.95, **kwargs):
        """
        Получаем n трендовых ключевых слов
        """
        
        keys = self.get_keywords(set_of_articles, **kwargs)
        
        keys, _ = self.cos_simularity(keys, threshold=0.95)

        return keys[:n]


    def get_embed(self, text):
        t = self.tokenizer(text.replace('\n', ''), padding=True, truncation=True, return_tensors='pt').to(self.device)
        with torch.no_grad():
            model_output = self.model.encoder(input_ids=t.input_ids, attention_mask=t.attention_mask, return_dict=True)
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu()


    def cos_simularity(self, keys_weights, threshold=0.95):
        """
        Считаем косинусную схожесть, возвращаем отсортированный по частоте список 
        объединённый ключей таблицу взаимной схожести
        """
        
        new_keys_weights = []

        cos_sim = np.ones((len(keys_weights), len(keys_weights)))
        
        cos = torch.nn.CosineSimilarity(dim=1)


        for i in range(len(keys_weights)):
            sim_keys = [keys_weights[i][0]]
            sim_weight = keys_weights[i][1]
            embed_i = torch.unsqueeze(self.get_embed(keys_weights[i][0]), 0)
            for j in range(i+1, len(keys_weights)):
                embed_2 = torch.unsqueeze(self.get_embed(keys_weights[j][0]), 0)

                cos_sim[i][j] = cos(embed_i, embed_2).numpy()[0]
                cos_sim[j][i] = cos_sim[i][j]

                if cos_sim[i][j] > threshold:
    
                    sim_keys.append((keys_weights[j][0]))
                    sim_weight += keys_weights[j][1]  

            new_keys_weights.append((sim_keys, [sim_weight]))  
        return sorted(new_keys_weights, key=lambda tup: tup[1], reverse=True), cos_sim

In [18]:
def get_trends(data, centroids_map, top_for_cluster=10, max_news_len=200):
    words_extractor = KeyWordsExtractor()

    top_articles_for_clusters = get_top_articles(data, centroids_map, top_k=top_for_cluster)
    
    
    trends_list = []
    pbar = tqdm(total=len(top_articles_for_clusters))
    pbar.set_description("Getting trends...")
    for top_articles in top_articles_for_clusters:
        top_articles = [' '.join(row.split(' ')[:max_news_len]) for row in top_articles]
        trends_list.append(words_extractor.get_trends(top_articles, threshold=0.95, top_p=1.0, max_length=256, min_length=5))
        pbar.update(1)

    pbar.close()

    return np.array(trends_list)

### Инсайты

In [19]:
def summarize(tokenizer, model, text,  n_words=None, compression=None, max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0):
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True)

    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def get_insites(data, centroids_map, top_for_cluster=10, max_news_len=200):
    t5_model = T5ForConditionalGeneration.from_pretrained('cointegrated/rut5-base-absum')
    t5_tokenizer = T5Tokenizer.from_pretrained('cointegrated/rut5-base-absum')

    top_articles_for_clusters = get_top_articles(data, centroids_map, top_k=top_for_cluster)
    
    
    insites_list = []
    pbar = tqdm(total=len(top_articles_for_clusters))
    pbar.set_description("Getting insites...")
    for top_articles in top_articles_for_clusters:
        top_articles = [' '.join(row.split(' ')[:max_news_len]) for row in top_articles]
        insites_list.append(summarize(t5_tokenizer, t5_model, ' '.join(top_articles)))
        pbar.update(1)

    pbar.close()

    return np.array(insites_list)

### Дайджест

In [20]:
def get_top_articles(data, centroids_map, top_k=5, text_col='text'):
    """
        Функция выделяет ближайших к центру top_k новостей для каждого кластера.
        Вход:
            data - данные о новостях с текстами и эмбеддингами,
            
    """
    top_texts_list = []

    for label, cluster_center in centroids_map.items():
        cluster = data[data['label'] == label]
        embeddings = list(cluster['embedding'])
        texts = list(cluster[text_col])
        
        distances = [euclidean_distances(cluster_center.reshape(1, -1), e.reshape(1, -1))[0][0] for e in embeddings]
        scores = list(zip(texts, distances))
        top_ = sorted(scores, key=lambda x: x[1])[:top_k]
        top_texts = list(zip(*top_))[0]
        top_texts_list.append(top_texts)
    return top_texts_list

def get_digest(data, centroids_map, top_clusters = 5):
    top_label = data['label'].value_counts()[:top_clusters].index.to_list()
    local_centroids = {label: centroids_map[label] for label in top_label}

    return np.array(get_top_articles(data, local_centroids, top_k=1, text_col='title'))

## Основное взаимодейтвие

### Утилиты

In [27]:
def get_data_by_period(df, start_date_string = None, end_date_string = None):
    """
        Выдление новостей для заданного временного промежутка.
        Вход:
            df - DataFrame с колонками 'content' и 'date'
            start_date_string - строка задающая начало временного периода, 
        в формате yyyy-mm-dd
            end_date_string - строка задающая конец временного периода.
        Выход:
            отфильтрованный по времени DataFrame

    """
    df.loc[:, 'date'] = pd.to_datetime(df['date'])
    mask = (df['date'] > pd.to_datetime(start_date_string)) & (df['date'] < pd.to_datetime(end_date_string))
    return df.loc[mask,:]

In [28]:
def format_output(json_response):
    format_out_json = json.loads(json_response)
    print('Дайджесты:')
    print('\n'.join([str(digest['id']+1) + '. ' + digest['content'] for digest in format_out_json['digest']]))

    print('\nТренды:')
    print('\n'.join(['\n'.join(trends['content']) for trends in format_out_json['trends']]))

    print('\nИнсайты:')
    print('\n'.join([str(insite['id']+1) + '. ' +insite['content'] for insite in format_out_json['insites']]))

    
def format_json_response(insites, trends, digest, start_date, end_date):
    result = {"dates": {"start_date": start_date, "end_date": end_date}, "insites": [], "trends": [], "digest": []}
    
    
    for insite, id in zip(insites, range(0, len(insites))):
        result["insites"].append({"id": id, "content": insite})

    for trend, id in zip(trends, range(0, len(trends))):
        result["trends"].append({'id' : id, "content": [trend[0][0][0], trend[1][0][0]]})
    
    for digest_el, id in zip(digest, range(0, len(digest))):
        result["digest"].append({'id' : id, "content": digest_el[0]})



    return json.dumps(result)

In [29]:
def download_data(url, file_name):
    gdown.download(url, file_name, fuzzy=True)

#скачивание данных для директоров
download_data(
    'https://drive.google.com/file/d/1-cupnhoEB_isDyO0CPMAnuamqvCnI4us/view?usp=sharing',
              'ceo_data.csv')
#скачивание данных для бухгалтеров
download_data( 
    'https://drive.google.com/file/d/1-rtHxQcfj6XJZJ2UWgO6gcmuLrBqn_8q/view?usp=sharing',
              'acc_data.csv')

Downloading...
From: https://drive.google.com/uc?id=1-cupnhoEB_isDyO0CPMAnuamqvCnI4us
To: /content/ceo_data.csv
100%|██████████| 21.2M/21.2M [00:00<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-rtHxQcfj6XJZJ2UWgO6gcmuLrBqn_8q
To: /content/acc_data.csv
100%|██████████| 33.1M/33.1M [00:00<00:00, 86.7MB/s]


### Получение ответа для бухгалтеров и директоров 

In [30]:
def get_response(input_data, start_date, end_date):
    """
        Получение инсайтов, трендов и дайджеста.
        Вход:
            df - DataFrame с колонками 'content' и 'date'
            start_date_string - строка задающая начало временного периода, 
        в формате yyyy-mm-dd
            end_date_string - строка задающая конец временного периода.
        Выход:
            кортеж из неотформатированных результатов (инсайты, тренды, дайджест)
    """
    text_pool = get_data_by_period(input_data, start_date, end_date) # выделение новостей из конкретного временного периода

    embeddins_pool = RuBertEmbedder().encode_data(text_pool, data_column='content') # получшение эмбеддингов из содержимого новостей

    clustering_data, centroids = KMeansClustering(text_pool, embeddins_pool).clustering() # кластеризация
    centroids_map = {i: centroids[i] for i in range(len(centroids))}

    insites = get_insites(clustering_data, centroids_map, top_for_cluster=30, max_news_len=100) # инсайты
    trends = get_trends(clustering_data, centroids_map, top_for_cluster=5, max_news_len=80) # тренды
    digest = get_digest(clustering_data, centroids_map, top_clusters=3) # дайджест

    response = format_json_response(insites, trends, digest, start_date, end_date) # агрегация результатов в формат Json

    return response

In [33]:
ceo_data = pd.read_csv('/content/ceo_data.csv')
ceo_json_response = get_response(ceo_data, '2022-02-14', '2022-02-28')

Encoding data...
Encoding done!
Clustering data...
Clustering done!


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [34]:
print("Директора")
format_output(ceo_json_response)

Директора
Дайджесты:
1. Российские биржи приостановили торги
2. ФАС разбертся, насколько обоснованным было резкое повышение цен в сети магазинов бытовой техники и электроники DNS.
3. У России есть наджный план на случай введения Западом новых санкций от драматичного сценария страну уберегут прочные макрофинансовые показатели, уверен министр финансов Антон Силуанов.

Тренды:
космический мусор
космонавтика
сбербанк
экосистема
платежи
финансы
управление продуктом
сбербанк
маркетплейс
ценообразование
сбер
банки

Инсайты:
1. Акции американской космической компании Virgin Galactic выросли на 13,11 до 8,85 за бумагу.
2. Российские биржи приостановили торги по всем видам ценных бумаг изза приближения котировок к установленным границам
3. Глава Госдумы Вячеслав Володин призвал россиян верить в рубль, если он падает
4. Эксперты по работе с персоналом крупных российских компаний обсудили, как изменилось отношение сотрудников к работе в компаниях за последние два года
5. Власти России прислушались

In [ ]:
print("Бухгалтеры")
format_output(acc_json_response)

In [ ]:
acc_data = pd.read_csv('/content/acc_data.csv')
acc_json_response = get_response(acc_data, '2022-02-14', '2022-02-28')